# How often do clocks need to be synchronised to have a deviation smaller than x?

In [16]:
ns2s = nanoseconds_to_seconds = lambda ns: (10 ** -9) * ns
ms2s = milliseconds_to_seconds = lambda ms: (10 ** -3) * ms
minutes = lambda m: 60 * m


In [17]:
def get_synchronization_frequenzy(maximum_deviation, max_clock_driftrate):
    # in the worst case one clock runs maximum fast and the other maximum slow, therefore 2 * driftrate
    synchronize_every_x_seconds = maximum_deviation / (2 * max_clock_driftrate)
    return round(1 / synchronize_every_x_seconds)


max_deviation_wanted = ns2s(50)
max_clock_driftrate = 10 ** -6
print(
    "Synchronisationen pro sekunde: ",
    get_synchronization_frequenzy(max_deviation_wanted, max_clock_driftrate),
)


Synchronisationen pro sekunde:  40


# Drift nach x Zeit

In [18]:
def get_deviation(ticks_of_good_clock_per_second, ticks_of_deviating_clock_per_second):
    return abs(ticks_of_good_clock_per_second - ticks_of_deviating_clock_per_second)


print(
    "drift nach einer minute ist: ",
    round(get_deviation(ms2s(1000), ms2s(950)) * minutes(1), 3),
)


drift nach einer minute ist:  3.0


# Offset zwischen zwei uhren

In [1]:
times = { # jede spalte im dict ist eine übertragung
    "t0": [13.420, 20.752, 25.325],
    "t1": [15.450, 22.00, 30.855],
    "t2": [25.700,24.925, 32.255],
    "t3": [24.925, 24.650, 31.455],
}


In [2]:
def get_time_offset(t0, t1, t2, t3):
    # t0 und t3 sind sender seite, t2 und t3 sind empfänger seite
    res = ((t1 - t0) + (t2 - t3)) / 2
    res=round(res, 4)
    print(f"time_offset: {t1} - {t0} + {t2} - {t3} = {res}")
    return res


offsets = []
for n in range(len(times["t0"])):
    offset = get_time_offset(
        times["t0"][n], times["t1"][n], times["t2"][n], times["t3"][n]
    )
    offsets.append(round(offset, 4))

offsets


time_offset: 15.45 - 13.42 + 25.7 - 24.925 = 1.4025
time_offset: 22.0 - 20.752 + 24.925 - 24.65 = 0.7615
time_offset: 30.855 - 25.325 + 32.255 - 31.455 = 3.165


[1.4025, 0.7615, 3.165]

# round trip delay

In [3]:
def get_round_trip_delay(t0, t1, t2, t3):
    res=(t3 - t0) - (t2 - t1)
    res = round(res,4)
    print(f"round trip delay: {t3} - {t0} + {t2} - {t1} = {res}")
    return res


delays = []
for n in range(len(times["t1"])):
    deviation = get_round_trip_delay(
        times["t0"][n], times["t1"][n], times["t2"][n], times["t3"][n]
    )
    delays.append(round(deviation, 4))

delays


round trip delay: 24.925 - 13.42 + 25.7 - 15.45 = 1.255
round trip delay: 24.65 - 20.752 + 24.925 - 22.0 = 0.973
round trip delay: 31.455 - 25.325 + 32.255 - 30.855 = 4.73


[1.255, 0.973, 4.73]

# offset


In [4]:
def get_time_shifted(t0,t1,t2,t3):
    offset=get_time_offset(t0,t1,t2,t3)
    delay=get_round_trip_delay(t0,t1,t2,t3)
    res=t3 + offset - delay/2
    res = round(res,4)
    print(f"time_shifted: {t3} + {offset} - {delay} / 2 = {res}")
    return (res,offset,delay)

In [9]:
time_offsets = []
for n in range(len(times["t0"])):
    print(f'Anfrage: Sender: {times["t0"][n]}, Empfänger: {times["t1"][n]} \nAntwort: Sender: {times["t2"][n]}, Empfänger: {times["t3"][n]}')
    time_offsets.append(
        get_time_shifted(times["t0"][n], times["t1"][n], times["t2"][n], times["t3"][n])
    )
    print()
    
print(f"die kleinste verzögerung wird gewählt also um {min(time_offsets[1])}")


Anfrage: Sender: 13.42, Empfänger: 15.45 
Antwort: Sender: 25.7, Empfänger: 24.925
time_offset: 15.45 - 13.42 + 25.7 - 24.925 = 1.4025
round trip delay: 24.925 - 13.42 + 25.7 - 15.45 = 1.255
time_shifted: 24.925 + 1.4025 - 1.255 / 2 = 25.7

Anfrage: Sender: 20.752, Empfänger: 22.0 
Antwort: Sender: 24.925, Empfänger: 24.65
time_offset: 22.0 - 20.752 + 24.925 - 24.65 = 0.7615
round trip delay: 24.65 - 20.752 + 24.925 - 22.0 = 0.973
time_shifted: 24.65 + 0.7615 - 0.973 / 2 = 24.925

Anfrage: Sender: 25.325, Empfänger: 30.855 
Antwort: Sender: 32.255, Empfänger: 31.455
time_offset: 30.855 - 25.325 + 32.255 - 31.455 = 3.165
round trip delay: 31.455 - 25.325 + 32.255 - 30.855 = 4.73
time_shifted: 31.455 + 3.165 - 4.73 / 2 = 32.255

die kleinste verzögerung wird gewählt also um 0.7615


time_offset: 15.45 - 13.42 + 25.7 - 24.925 = 1.4025
round trip delay: 24.925 - 13.42 + 25.7 - 15.45 = 1.255
time_shifted: 24.925 + 1.4025 - 1.255 / 2 = 25.7

time_offset: 22.0 - 20.752 + 24.925 - 24.65 = 0.7615
round trip delay: 24.65 - 20.752 + 24.925 - 22.0 = 0.973
time_shifted: 24.65 + 0.7615 - 0.973 / 2 = 24.925

time_offset: 30.855 - 25.325 + 32.255 - 31.455 = 3.165
round trip delay: 31.455 - 25.325 + 32.255 - 30.855 = 4.73
time_shifted: 31.455 + 3.165 - 4.73 / 2 = 32.255

die kleinste verzögerung wird gewählt also um 0.7615

# GPS


In [37]:
C = 3 * (10 ** 5) # Lichtgeschwindigkeit in km/s
C2 = C / 1000 # Lichtgeschwindigkeit in m/mu_s

T_A = "t_A"
T_B = "t_B"
entfernung = 3
messungen = {1: {T_A: 7, T_B: 5}, 2: {T_A: 2, T_B: -6}, 3: {T_A: -1019, T_B: -1011}}


In [42]:
3000/C2

10.0

In [47]:
def messe_distanz(t_A, t_B, distanz_zwischen_A_und_B_in_m):
    sendezeit_A_nach_B_in_mu_s = distanz_zwischen_A_und_B_in_m / C2
    delta = (t_A + t_B - sendezeit_A_nach_B_in_mu_s) / 2 # uhr abweichung
    
    entfernung_von_A = C2 * (t_A - delta)
    print(f"delta: {delta}, entfernung zu A: {entfernung_von_A}")


for m in messungen:
    messe_distanz(messungen[m][T_A], messungen[m][T_B], 3000)


delta: 1.0, entfernung zu A: 1800.0
delta: -7.0, entfernung zu A: 2700.0
delta: -1020.0, entfernung zu A: 300.0
